In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from wordcloud import WordCloud

In [48]:
data = pd.read_csv('datasets/DataAnalyst.csv')

In [49]:
data.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True


#### Data cleaning

###### remove unnamed column

In [50]:
data.drop(['Unnamed: 0'], axis=1,inplace=True)

###### check the missing values

In [51]:
def missing_values_table(df):
    # total missing value
    mis_val = df.isnull().sum()

    #percentage of missing value
    mis_val_percent = 100*df.isnull().sum()/len(df)

    #make a table with the results
    mis_val_table = pd.concat([mis_val,mis_val_percent], axis=1)

    # rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0:'Missing Values',
                1:'% of Total Values'}
    )

    # sort the table by percentage of missing descending
    mis_val_table_ren_columns=mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1]!=0].sort_values(
            '% of Total Values', ascending=False).round(1)
    
    
    # print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")

    # return the dataframe with missing information
    return mis_val_table_ren_columns

In [52]:
missing_values_table(data)

Your selected dataframe has 15 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
Company Name,1,0.0


In [53]:
data['Easy Apply'].value_counts()

Easy Apply
-1      2173
True      80
Name: count, dtype: int64

In [54]:
data['Competitors'].value_counts()

Competitors
-1                                                                 1732
Robert Half, Insight Global                                          14
Adecco, Manpower                                                     14
Artech Information Systems, Mindlance, Tech Mahindra                 10
Google, Microsoft, Samsung Electronics                               10
                                                                   ... 
Carr Riggs & Ingram, Cherry Bekaert, LBMC                             1
Transnational Foods, One Source Distributors                          1
GE                                                                    1
Fiserv, First Data, Jack Henry & Associates                           1
Centura Health, HealthONE, Denver Health and Hospital Authority       1
Name: count, Length: 291, dtype: int64

In [55]:
data['Rating'].value_counts()[:5]

Rating
-1.0    272
 3.9    140
 3.6    135
 3.7    132
 5.0    131
Name: count, dtype: int64

###### As some of the columns contains -1 or '-1.0' or '-1' etc . We need to clean this(This is kind of null values)

In [56]:
# Replace -1 or -1.0 or '-1' to NaN

data=data.replace(-1,np.nan)
data=data.replace(-1.0,np.nan)
data=data.replace('-1',np.nan)

In [57]:
missing_value_table(data)

Your selected dataframe has 15 columns.
There are 12 columns that have missing values.


,Missing Values,% of Total Values
Easy Apply,2173,96.4
Competitors,1732,76.9
Founded,660,29.3
Industry,353,15.7
Sector,353,15.7
Rating,272,12.1
Headquarters,172,7.6
Size,163,7.2
Type of ownership,163,7.2
Revenue,163,7.2


###### Now you can see there are lots of missing values in the dataset. 'Easy Apply' and 'Competitors' has maximum number of missing values (> 50%)

###### Most companies has null values in easy apply because they are not hiring.

In [58]:
data['Company Name'].head()

0             Vera Institute of Justice\n3.2
1    Visiting Nurse Service of New York\n3.8
2                           Squarespace\n3.4
3                              Celerity\n4.1
4                               FanDuel\n3.9
Name: Company Name, dtype: object

In [59]:
data['Company Name'] = data['Company Name'].str.extract(r'(.+?)\n')

In [60]:
data[['Job Title', 'Department']] = data['Job Title'].str.extract(r'(.+?),\s*(.*)')

In [61]:
data[['Job Title', 'Department']].head()

,Job Title,Department
0,Data Analyst,Center on Immigration and Justice (CIJ)
1,NaN,NaN
2,Senior Data Analyst,Insights & Analytics Team [Customer Operations]
3,NaN,NaN
4,NaN,NaN


In [68]:
data['Salary Estimate'] = data['Salary Estimate'].str.extract(r'\$(\d+K)-\$(\d+K)').iloc[:, 0]

In [70]:
data['Salary Estimate']

0       37K
1       37K
2       37K
3       37K
4       37K
       ... 
2248    78K
2249    78K
2250    78K
2251    78K
2252    78K
Name: Salary Estimate, Length: 2253, dtype: object